# Basic Computation

In this lesson, we discuss how to do scientific computations with xarray
objects. Our learning goals are as follows. By the end of the lesson, we will be
able to:

- Apply basic arithmetic and numpy functions to xarray DataArrays / Dataset.
- Use Xarray's label-aware reduction operations (e.g. `mean`, `sum`) weighted
  reductions.
- Apply arbitrary functions to Xarray data via `apply_ufunc`.
- Use Xarray's broadcasting to compute on arrays of different dimensionality.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Ask Xarray to not show data values by default
xr.set_options(display_expand_data=False)

%config InlineBackend.figure_format='retina'

## Example Dataset

First we load a dataset. We will use the
[NOAA Extended Reconstructed Sea Surface Temperature (ERSST) v5](https://www.ncei.noaa.gov/products/extended-reconstructed-sst)
product, a widely used and trusted gridded compilation of of historical data going back to 1854.

In [ ]:
ds = xr.tutorial.load_dataset("ersstv5")
ds

Let's do some basic visualizations of the data, just to make sure it looks
reasonable.


In [ ]:
ds.sst.isel(time=0).plot(vmin=-2, vmax=30);

## Arithmetic

Xarray dataarrays and datasets work seamlessly with arithmetic operators and
numpy array functions.

For example, imagine we want to convert the temperature (given in Celsius) to
Kelvin:


In [ ]:
sst_kelvin = ds.sst + 273.15
sst_kelvin

The dimensions and coordinates were preserved following the operation.

<div class="alert alert-warning">
    <strong>Warning:</strong> Although many xarray datasets have a <code>units</code> attribute, which is used in plotting,
    Xarray does not inherently understand units. However, xarray can integrate with <a href="https://pint.readthedocs.io/en/0.12/">pint</a>, which provides full unit-aware operations. See <a href="https://pint-xarray.readthedocs.io">pint-xarray</a> for more.
</div>


## Applying functions

We can apply more complex functions to Xarray objects.
Imagine we wanted to compute the following expression as a function of SST
($\Theta$) in Kelvin:

$$ f(\Theta) =  0.5 \ln(\Theta^2) $$


In [ ]:
f = 0.5 * np.log(sst_kelvin**2)
f

## Applying Arbitrary Functions

It's awesome that we can call `np.log(ds)` and have it "just work". However, not
all third party libraries work this way.

numpy's [nan_to_num](https://numpy.org/devdocs/reference/generated/numpy.nan_to_num.html) for example will return a numpy array

In [ ]:
np.nan_to_num(ds.sst, 0)

It would be nice to keep our dimensions and coordinates. 

We can accomplish this with [xr.apply_ufunc](https://docs.xarray.dev/en/stable/generated/xarray.apply_ufunc.html#xarray.apply_ufunc)

In [ ]:
xr.apply_ufunc(np.nan_to_num, ds.sst, 0)

```{tip}
`apply_ufunc` is a powerful function. It has many options for doing more complicated things. Unfortunately, we don't have time to go into more depth here. See the [`apply_ufunc` tutorial material](https://tutorial.xarray.dev/advanced/apply_ufunc/apply_ufunc.html) for more.
```


## Reductions

Reductions are functions that reduce the dimensionlity of our dataset. For example taking the mean sea surface temperature along `time` of our 3D data, we "reduce" the  `time` dimension and are left with a 2D array.

Just like in numpy, we can reduce xarray DataArrays along any number of axes.

In [ ]:
sst = ds.sst
sst.mean(axis=0)

However, rather than performing reductions by specifying `axis` (as in numpy), we can instead perform
them using _dimension names_. This turns out to be a huge convenience, particularly in
complex calculations it can be hard to remember which axis corresponds to 
which dimension name:

In [ ]:
sst.mean(dim="time")

You can reduce over multiple dimensions

In [ ]:
sst.mean(["lat", "time"])

If no dimension is specified, the reduction is applied across all dimensions.

In [ ]:
sst.mean()

All of the standard numpy reductions (e.g. `min`, `max`, `sum`, `std`, etc.) are
available on both [Datasets](https://docs.xarray.dev/en/stable/api.html#aggregation) and [DataArrays](https://docs.xarray.dev/en/stable/api.html#id6).


```{exercise}
:label: sst-mean
Take the mean of `sst` in both longitude and latitude. Make a simple timeseries plot.
```
````{solution} sst-mean
:class: dropdown

sst.mean(["lat", "lon"]).plot();
```
